<a href="https://colab.research.google.com/github/Amsterdam-Internships/Readability-Lexical-Simplification/blob/master/Generating_Dutch_Sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Sentences for Dutch Evaluation

## Installation

In [ ]:
!pip install spacy==3.2.0
!pip install python-docx 

     |████████████████████████████████| 6.0 MB 6.6 MB/s 
     |████████████████████████████████| 653 kB 40.9 MB/s 
     |████████████████████████████████| 457 kB 43.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 181 kB 57.4 MB/s 
     |████████████████████████████████| 10.1 MB 46.3 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

In [ ]:
import docx
import glob
import spacy 
import nltk

import os
from docx import Document
from random import sample


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!python -m spacy download nl_core_news_sm

     |████████████████████████████████| 17.4 MB 272 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')


In [ ]:
nlp = spacy.load("nl_core_news_sm")

## Loading Data

In [ ]:
# Loading in the moeilijke woorden list
with open ("/content/moeilijke woorden.txt","r") as infile:
  data = infile.readlines()

complex_words = []

for line in data:
  if line.startswith("["):
    continue
  if line.count(" ")>0:
    continue
  line = line.strip()
  if len(line) <2:
    continue

  line = line.replace("(","")
  line = line.replace(")","")
  complex_words.append(line)

print(len(complex_words))

488


In [ ]:
# All texts into a list of paragraphs
paragraphs = []
i = 0
path = '/content/drive/MyDrive/Thesis/code/Preprocessing/Data for Iva'
for filename in glob.glob(os.path.join(path, '*.docx')):
  doc = Document(filename)
  i+=1
  for para in doc.paragraphs:
    text = para.text.strip().replace("\n"," ")
    if len(text)>0:
      paragraphs.append(text)

In [ ]:
print(i)

48


## Preprocessing the Texts

In [ ]:
# https://stackoverflow.com/questions/67500193/cleaning-lemmatizing-dutch-dataset-using-spacy
# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists

def lemmatizer(texts):
    texts = [text.replace("\n", "").strip() for text in texts]
    token_doc = nlp.pipe(texts)
    lemma_doc = nlp.pipe(texts)
    all_tokens = [[y.text for y in x] for x in token_doc]
    all_lemmas = [[y.lemma_ for y in x] for x in lemma_doc]
    
    flattened_cleaned_lemmas = [item for sublist in all_lemmas for item in sublist]
    flattend_all_toks = [item for sublist in all_tokens for item in sublist] 

    return flattened_cleaned_lemmas, flattend_all_toks
text = ['duurzame dingen']
lemmatizer(text)

(['duurzaam', 'ding'], ['duurzame', 'dingen'])

In [ ]:
all_sents = 0
sents = []
sents_lemmas = []
sents_tokens = []

for paragraph in paragraphs:

  doc = nlp(paragraph)
  
  for sent in doc.sents:
    all_sents += 1

    # Some length filters
    if len(sent)>4 and not len(sent) > 20:
      contains_verb = False

    # To filter out headers: at least one verb     
      for tok in sent:

        if tok.pos_ =="VERB":
          contains_verb = True  
          break
    
      if contains_verb == True:
        sent = str(sent).replace("\t","")
        sents.append(sent)

        sent_lemma, sent_tokens = lemmatizer([str(sent)])
        sents_lemmas.append(sent_lemma)
        sents_tokens.append(sent_tokens)
        

In [ ]:
print(all_sents)
print(len(sents))

15334
6084


## Only sentences that contain one complex word

In [ ]:
complex_lemmas = lemmatizer(complex_words)[0]  

In [ ]:
complex_sents = []
corresponding_cwords = []
corresponding_clemmas= []

for sent, lem_sent, tok_sent in zip(sents, sents_lemmas,sents_tokens):
  print(sent)
  found_lemmas = []
  found_index = []


  for complex_lemma in complex_lemmas:
    if complex_lemma in lem_sent:
      found_lemmas.append(complex_lemma)
      found_index = lem_sent.index(complex_lemma)

    # Multiple complex words in the sentence, we don't want that
    if len(found_lemmas)>1:
      break

  # If there is exactly one complex word in the sentence, we add it to the list
  if len(found_lemmas)==1:
    complex_sents.append(sent)
    corresponding_clemmas.append(found_lemmas[0])
    corresponding_cwords.append(tok_sent[found_index])          

Streaminguitvoer ingekort tot de laatste 5000 regels.
Je krijgt les van een kunstvakdocent.
start 14 september 2019 en 11 januari 2020
Het zal niet lang duren voordat ook je kind zich als een visje in het water voelt.
T 020 665 0811 | sportfondsenbadamsterdamoost.nl
In het eerste levensjaar leren baby’s meer dan in de rest van hun leven.
Ouders krijgen in deze les allerlei informatie over de ontwikkelingsstadia die hun baby doormaakt.
Samen spelen, ontdekken, leren en bovenal plezier maken met je baby.
De complete babycursus Baby Sensory, uitgeroepen tot beste ontwikkelingsprogramma uit Engeland, is nu in Amsterdam te volgen.
Kijk voor de openingsdagen en -tijden op de website van Baby Sensory.
(Alleen starten bij aanvang blok)
Een veilige en plezierige omgeving waar dreumesen en peuters kunnen ontdekken en zich kunnen ontwikkelen door middel van spel.
Een kans voor ouders om samen met hun peuter/ kleuter het avontuur aan te gaan.
Elke les is samengesteld om de ontwikkeling van de moto

In [ ]:
import random
random.seed(3)
clemma_sample = random.sample(complex_lemmas, 250)

In [ ]:
selected_words = []
selected_sents = []

for i, c_lemma in enumerate(clemma_sample):
  # Leaving room to manually remove 10 sentences
  if len(selected_sents)==110:
    break
  
  for sent, cor_clemma, cor_cword in zip(complex_sents, corresponding_clemmas, corresponding_cwords):
    if c_lemma == cor_clemma:
      # print(c_lemma)
      # print(cor_clemma)
      # print(cor_cword)
      selected_words.append(cor_cword)
      selected_sents.append(sent)
      break
print(len(selected_words))
print(i)
print(len(complex_lemmas))

108
249
488


In [ ]:
for word, sent in zip(selected_words, selected_sents):
  print(word, sent)

middels In 2018 zijn er 419 personen in detentie voorgelicht middels een groepsvoorlichting.
fungeert De openbare ruimte fungeert als het bindende element met meanderende  fiets- en wandelpaden.
opgave Het uitganspunt dat we denken, werken en handelen vanuit een gezamenlijk opgave.
uitgezet In de planning zijn deze stappen in de tijd uitgezet.
momenteel Dienstverlening zit momenteel in een belangrijke fase.
gerefereerd Er wordt hierbij ook gerefereerd aan het Jongerenservicepunt in Nieuw-West dat helaas een aantal jaar geleden is opgeheven.
wederom Na deze datum heeft wederom een beoordeling plaatsgevonden en zijn 5 gedoogverklaringen verlengd tot maart 2022.
innovatieve De nieuwbouw gaat in vormgeving en innovatieve bouwmethoden mee in de historische ontwikkelingen in het park.
verblijfskwaliteit De bebouwing aan de Klaprozenweg is zeer wisselend en de straat heeft geen verblijfskwaliteit.
categorieën Wij gebruiken verschillende categorieën persoonsgegevens van de medewerkers::
overhe

In [ ]:
training_sents = []
for sent, word in zip(complex_sents, corresponding_cwords):
  if sent not in selected_sents:
    training_sents.append(sent+"\t"+word)


In [ ]:
print(len(training_sents))
len(training_sents) 
print(len(selected_sents))
print(len(complex_sents))

1026
108
1137


In [ ]:
print(training_sents)

['Werk- en privéomstandigheden (duurzame inzetbaarheid en vitaliteit)\tduurzame', 'Denk aan het verder ontwikkelen van jouw persoonlijke effectiviteit, gesprekstechnieken of taalontwikkeling.\tontwikkelen', 'De bedoeling van leren en ontwikkelen is:\tontwikkelen', 'Wellicht bemerk je je bij deze ‘inzoom’ dat er ook teamontwikkeling nodig is – toets\tWellicht', 'Het carrièrecentrum geeft advies over mobiliteit, loopbaanontwikkeling en duurzame inzetbaarheid.\tduurzame', 'De diversiteit verrijkt ook de oplossingen.\tdiversiteit', 'De gemeente Amsterdam stimuleert mobiliteit en heeft daarbij ook aandacht voor de mogelijkheden van promotie en demotie.\tstimuleert', 'Onder mobiliteit verstaan wij elke verandering in status en werkcontext, met een concrete verandering van werkinhoud en verantwoordelijkheid.\tconcrete', 'Door medewerkers op een lager niveau met minder verantwoordelijkheden te plaatsen, wordt bijgedragen aan hun duurzame inzetbaarheid.\tduurzame', 'prestatie met bijbehorend st

## Writing out the data

In [ ]:
with open ("/content/sents_for_dutch_eval.txt", "w", encoding="utf-8") as outfile:
  for sent, word in zip(selected_sents, selected_words):
    outfile.write(f"{sent}\t{word}\tannotations\n")

In [ ]:
with open ("/content/dutch_train_sents.txt", "w", encoding="utf-8") as outfile:
  for line in training_sents:
    outfile.write(line+"\n")